# This notebook takes the raw excel file (output from HAWC2) to generate a new excel containing the channels of interest (fatigue loads).

In [ ]:
import pandas as pd
import numpy as np

$\color{red}{\text{-Check why are the values of rotor speed so high}}$

$\color{red}{\text{-Which M exponent for Mz shaft }}$



In [59]:
%%time 
df = pd.read_excel('ChannelStatistics_DTU10MW_ParStudyUniformParSpace_1hSims_Set2_Parts.xlsx')

Wall time: 10min 3s


# 1 - Select just the columns of interest from the dataframe


In [60]:
# 64 : Ch 10 Rotor speed Mean 
# 71 : Ch 11 Aero rotor torque Mean 
# 78 : Ch 12 Aero rotor power Mean 
# 85 : Ch 13 Aero rotor thrust Mean 
# 603 : Ch 87 DLL : 2 inpvec :  2  pelec w Mean 
# 596 : Ch 86 DLL : 2 inpvec :  1  mgen lss [nm] Mean 
# 618 : pointno
auxiliaryChs = np.array([64,71,78,85,603,596])

# 0 : Filename
# 159 : Ttop FA M4
# 166 : Ttop SS M4 
# 201 : TBase FA M4
# 208 : TBase SS M4 
# 257 : Ttop Yaw M4
# 300 : Mz shaft M8   FIXME: Which M to use?
# 434 : BRM Flap M12
# 441 : BRM Edge M12
loadsChs = np.array([0,159,166,201,208,257,300,434,441])

FinalIdxs = np.concatenate([loadsChs,auxiliaryChs])

In [61]:
df_clean = df.iloc[:,FinalIdxs]
df_clean.head(3)

,Filename,Ch 23 MomentMx Mbdy:towertop nodenr: 1 coo: tower tower top -1: below top mass SeqM4,Ch 24 MomentMy Mbdy:towertop nodenr: 1 coo: tower tower top -1: below top mass SeqM4,Ch 29 MomentMx Mbdy:tower nodenr: 1 coo: tower tower base SeqM4,Ch 30 MomentMy Mbdy:tower nodenr: 1 coo: tower tower base SeqM4,Ch 37 MomentMz Mbdy:towertop nodenr: 1 coo: towertop yaw bearing SeqM4,Ch 43 MomentMz Mbdy:shaft nodenr: 4 coo: shaft main bearing SeqM8,Ch 62 MomentMx Mbdy:blade1 nodenr: 1 coo: blade1 blade 1 node 1 SeqM12,Ch 63 MomentMy Mbdy:blade1 nodenr: 1 coo: blade1 blade 1 node 1 SeqM12,Ch 10 Rotor speed Mean,Ch 11 Aero rotor torque Mean,Ch 12 Aero rotor power Mean,Ch 13 Aero rotor thrust Mean,Ch 87 DLL : 2 inpvec : 2 pelec w Mean,Ch 86 DLL : 2 inpvec : 1 mgen lss [nm] Mean
0,dtu_10mw_pseudomc_unifspace_set2_pointno1_s110...,713384810,195165854,6489319366,8915184961,767357182,89035564,4132305498,15025672875,523602,1181489786,618640195,221697183,574561800852,-1167359689254
1,dtu_10mw_pseudomc_unifspace_set2_pointno1_s110...,685115150,188637533,6169354068,8343460046,771105023,55045064,4107964815,15011208161,523603,1211847873,634536256,224971361,589705690041,-1198128134071
2,dtu_10mw_pseudomc_unifspace_set2_pointno1_s110...,697356910,190147086,6340238278,8418374055,769678695,57773016,4120478089,15008259523,523601,1220928166,639287684,225924356,594152270828,-1207168128500


# 2- Find missing points

In [62]:
pts = []
for fname in df_clean.Filename.values:
    pts.append(int(fname.split('pointno')[1].split('_')[0]))

In [63]:
%%time
# 41.3 s
miss_pts = []
for i in range(1,10000):
    occ_count = np.count_nonzero(np.array(pts) == i)
    
    if occ_count != 6:
        miss_pts.append(i)

Wall time: 51.9 s


In [64]:
# Set 2
print(f'{len(miss_pts)} missing points: {miss_pts}')

21 missing points: [33, 315, 525, 993, 1665, 3765, 4425, 6165, 7425, 7444, 7445, 7446, 7447, 7448, 7449, 7450, 7451, 7452, 7453, 7454, 7455]


In [20]:
# Set 1
print(f'{len(miss_pts)} missing points: {miss_pts}')

4 missing points: [945, 5121, 5145, 9345]


In [73]:
# Short sanity check
assert (60000-len(df))/6 == len(miss_pts) , 'Review misisng point (some points might not have 6 parts)'

# 3 - Add the pointno as a column (for inner join with enviromental inputs afterwards)

In [21]:
def categorise(row):
    pt_nr = int(row['Filename'].split('pointno')[1].split('_')[0])
    return pt_nr

In [22]:
df_clean['pointno'] = df_clean.apply(lambda row: categorise(row), axis=1)

<ipython-input-22-906899d4fbb8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['pointno'] = df_clean.apply(lambda row: categorise(row), axis=1)


In [23]:
df_clean.head(2)

,Filename,Ch 23 MomentMx Mbdy:towertop nodenr: 1 coo: tower tower top -1: below top mass SeqM4,Ch 24 MomentMy Mbdy:towertop nodenr: 1 coo: tower tower top -1: below top mass SeqM4,Ch 29 MomentMx Mbdy:tower nodenr: 1 coo: tower tower base SeqM4,Ch 30 MomentMy Mbdy:tower nodenr: 1 coo: tower tower base SeqM4,Ch 37 MomentMz Mbdy:towertop nodenr: 1 coo: towertop yaw bearing SeqM4,Ch 43 MomentMz Mbdy:shaft nodenr: 4 coo: shaft main bearing SeqM8,Ch 62 MomentMx Mbdy:blade1 nodenr: 1 coo: blade1 blade 1 node 1 SeqM12,Ch 63 MomentMy Mbdy:blade1 nodenr: 1 coo: blade1 blade 1 node 1 SeqM12,Ch 10 Rotor speed Mean,Ch 11 Aero rotor torque Mean,Ch 12 Aero rotor power Mean,Ch 13 Aero rotor thrust Mean,Ch 87 DLL : 2 inpvec : 2 pelec w Mean,Ch 86 DLL : 2 inpvec : 1 mgen lss [nm] Mean,pointno
0,dtu_10mw_pseudomc_unifspace_set1_pointno1_s100...,760289174,205473154,6808190138,9237842357,822160420,89477784,4277853226,15013140341,523606,1196777704,626646532,223566581,581934886396,-1182338031144,1
1,dtu_10mw_pseudomc_unifspace_set1_pointno1_s100...,749493777,199441295,6873877856,8770447018,817449089,63139847,4254422333,15005230148,523603,1230248192,644167046,227039098,598610906651,-1216228744380,1
2,dtu_10mw_pseudomc_unifspace_set1_pointno1_s100...,746513826,200588381,6875569177,8789839954,814138098,56485337,4250412389,15016339533,523604,1234657321,646477017,227490318,600783468506,-1220640372589,1
3,dtu_10mw_pseudomc_unifspace_set1_pointno1_s100...,744344551,200296712,6624168793,8801945320,827607135,57254745,4250064046,14997868127,523601,1226127505,642007837,226602793,596579082038,-1212103494486,1
4,dtu_10mw_pseudomc_unifspace_set1_pointno1_s100...,744142162,201723892,6516074947,8888929046,823596469,57250333,4254506338,15018660302,523602,1228455941,643227798,226813783,597716364421,-1214412383536,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59971,dtu_10mw_pseudomc_unifspace_set1_pointno10000_...,2488895856,237960404,8424245224,4433474218,2319619125,278752127,23649930794,16088275110,1004929,10537774401,10588681565,604572642,9999994630311,-10587211869976,10000
59972,dtu_10mw_pseudomc_unifspace_set1_pointno10000_...,2529522561,254396502,8362514733,5618646027,2325780088,278893190,23598161672,16056852453,1004889,10539430375,10589952348,598721892,9999993274792,-10587616017075,10000
59973,dtu_10mw_pseudomc_unifspace_set1_pointno10000_...,2561029478,270710360,8495680434,6845839932,2376808557,284747896,23518285972,16103462368,1004808,10541587394,10591393293,602814380,9999997529036,-10588352330171,10000
59974,dtu_10mw_pseudomc_unifspace_set1_pointno10000_...,2490221391,289223257,8663406398,7634917454,2281374217,310913453,23535394825,16138884170,1005475,10529973706,10586516914,600979357,9999990668026,-10581505469172,10000


### 4- Rename channels

In [37]:
df_clean.columns = ['File_name','TT_Mx','TT_My','TB_Mx','TB_My','TT_Mz','MS_Mz','BR_Mx','BR_My',
                    'Rot_speed','Ae_torque','Ae_power','Ae_thrust','P_ele','M_gen_lss','pointno']
for i,name in enumerate(df_clean.columns):
    print((i,name))

(0, 'File_name')
(1, 'TT_Mx')
(2, 'TT_My')
(3, 'TB_Mx')
(4, 'TB_My')
(5, 'TT_Mz')
(6, 'MS_Mz')
(7, 'BR_Mx')
(8, 'BR_My')
(9, 'Rot_speed')
(10, 'Ae_torque')
(11, 'Ae_power')
(12, 'Ae_thrust')
(13, 'P_ele')
(14, 'M_gen_lss')
(15, 'pointno')


# 5 - Take the average values for each simulation.

In [38]:
keys = []
vals = []
for col in df_clean.columns:
    if col != 'File_name':
        keys.append(col)
        vals.append(np.mean(df_clean[col].values.reshape(-1, 6),axis = 1))
        
loads_dict = dict(zip(keys, vals))

In [40]:
loads_df = pd.DataFrame(loads_dict)

# 6 - Load the enviromental variables (inputs) and remove unnecessary columns

In [41]:
df_HaltonSeq = pd.read_excel('Uniform6D_HaltonSequenceTable10000p_mod.xlsx')

In [42]:
df_HaltonSeq.head(3)

,Row,Sample_No,F_U,F_SigmaU,F_Alpha,F_MannL,F_MannGamma,F_Veer,U,SigmaU,Alpha,MannL,MannGamma,MannAlphaEpsilon,VeerDeltaPhi,TI,SigmaU_SigmaV_ratio,SigmaU_SigmaW_ratio
0,./shear/DTU10MW_UserShear_UniDatabase_PointNo1...,1,0.00,0.000000,0.0,0.000000,0.000000,0.000000,4.000000,0.100000,-0.650000,7.500000,1.000000,0.003469,-22.250000,0.025000,0.950504,0.944221
1,./shear/DTU10MW_UserShear_UniDatabase_PointNo2...,2,0.50,0.333333,0.2,0.142857,0.090909,0.076923,10.150758,1.208656,-0.139692,48.470634,1.363636,0.133573,-4.771217,0.119071,0.939917,0.913046
2,./shear/DTU10MW_UserShear_UniDatabase_PointNo3...,3,0.25,0.666667,0.4,0.285714,0.181818,0.153846,6.813467,2.261463,0.129071,88.379710,1.727273,0.316794,-3.981593,0.331911,0.955786,0.903031


In [46]:
# drop unnecessary columns 
env_df = df_HaltonSeq.drop(['Row','Sample_No','F_U','F_SigmaU','F_Alpha','F_MannL','F_MannGamma','F_Veer',
                           'SigmaU_SigmaV_ratio','SigmaU_SigmaW_ratio'], axis=1)

In [47]:
# add 'pointno' column for innerjoin with loads channels
env_df['pointno'] = np.arange(1,10001)

In [48]:
env_df.head(2)
# note that density is not included

,U,SigmaU,Alpha,MannL,MannGamma,MannAlphaEpsilon,VeerDeltaPhi,TI,pointno
0,4.000000,0.100000,-0.650000,7.500000,1.000000,0.003469,-22.250000,0.025000,1
1,10.150758,1.208656,-0.139692,48.470634,1.363636,0.133573,-4.771217,0.119071,2


# 7 - Merge (inner join) loads and enviromental dataframes into one.

In [50]:
set1_df = pd.merge(env_df,loads_df, how = 'inner', on='pointno') 

In [51]:
set1_df.set_index('pointno', inplace=True )

In [53]:
set1_df.head(2)

,U,SigmaU,Alpha,MannL,MannGamma,MannAlphaEpsilon,VeerDeltaPhi,TI,TT_Mx,TT_My,...,TT_Mz,MS_Mz,BR_Mx,BR_My,Rot_speed,Ae_torque,Ae_power,Ae_thrust,P_ele,M_gen_lss
pointno,,,,,,,,,,,,,,,,,,,,,
1,4.000000,0.100000,-0.650000,7.500000,1.000000,0.003469,-22.250000,0.025000,7.478346e+08,2.014445e+08,...,8.211800e+08,6.380933e+07,4.258895e+09,1.500923e+10,523604.000000,1.226237e+09,6.420682e+08,2.266344e+08,5.965992e+11,-1.212138e+12
2,10.150758,1.208656,-0.139692,48.470634,1.363636,0.133573,-4.771217,0.119071,3.576390e+09,7.044477e+08,...,3.750994e+09,1.420377e+09,1.072776e+10,1.627160e+10,842980.333333,9.793625e+09,8.301564e+09,1.255444e+09,7.772750e+12,-9.755099e+12


# 8 - Create the excel.

In [54]:
set1_df.to_excel('Set1.xlsx') ## Create excel 